# Thermophysical modeling (TPM) of asteroid Ryugu

## Load packages

Install the necessary packages only for the first time.

In [ ]:
# using Pkg
# Pkg.add("SPICE")
# Pkg.add("Downloads")
# Pkg.add("StaticArrays")
# Pkg.add("Rotations")
# Pkg.add("CairoMakie")

# Pkg.add(url="https://github.com/Astroshaper/AsteroidThermoPhysicalModels.jl")

In [ ]:
import AsteroidThermoPhysicalModels
import SPICE

using Downloads
using StaticArrays
using Rotations
using CairoMakie

include("../plot_shape.jl");

## Download necessary files
- SPICE kernels
- Shape model

In [ ]:
paths_kernel = [
    "lsk/naif0012.tls",
    "pck/hyb2_ryugu_shape_v20190328.tpc",
    "fk/hyb2_ryugu_v01.tf",
    "spk/2162173_Ryugu.bsp",
]

paths_shape = [
    "SHAPE_SFM_49k_v20180804.obj",
]

for path_kernel in paths_kernel
    url_kernel = "https://data.darts.isas.jaxa.jp/pub/hayabusa2/spice_bundle/spice_kernels/$(path_kernel)"
    filepath = joinpath("kernel", path_kernel)
    mkpath(dirname(filepath))
    isfile(filepath) || Downloads.download(url_kernel, filepath)
end

for path_shape in paths_shape
    url_shape = "https://data.darts.isas.jaxa.jp/pub/hayabusa2/paper/Watanabe_2019/$(path_shape)"
    filepath = joinpath("shape", path_shape)
    mkpath(dirname(filepath))
    isfile(filepath) || Downloads.download(url_shape, filepath)
end

## Load SPICE kernels

In [ ]:
for path_kernel in paths_kernel
    filepath = joinpath("kernel", path_kernel)
    SPICE.furnsh(filepath)
end

Prepare ephemerides for TPM

In [ ]:
P = SPICE.convrt(7.63262, "hours", "seconds")  # Rotation period of Ryugu

ncycles = 5  # Number of cycles to perform TPM
nsteps_in_cycle = 360  # Number of time steps in one rotation period

et_begin = SPICE.utc2et("2018-07-01T00:00:00")  # Start time of TPM
et_end   = et_begin + P * ncycles  # End time of TPM
et_range = range(et_begin, et_end; length=nsteps_in_cycle*ncycles+1);

In [ ]:
"""
Ephemerides data for input into TPM.

- `time` : Ephemeris times
- `sun`  : Sun's position in the RYUGU_FIXED frame
"""
ephem = (
    time = collect(et_range),
    sun  = [SVector{3}(SPICE.spkpos("SUN", et, "RYUGU_FIXED", "None", "RYUGU")[1]) * 1000 for et in et_range],
);

Clear the SPICE kernels

In [ ]:
SPICE.kclear()

# Load a shape model
The OBJ format is only supported.

In [ ]:
path_obj = joinpath("shape", "ryugu_test.obj")  # Small model for test
# path_obj = joinpath("shape", "SHAPE_SFM_49k_v20180804.obj")

In [ ]:
shape = AsteroidThermoPhysicalModels.load_shape_obj(path_obj; scale=1000, find_visible_facets=true)

# TPM setup and execution

Thermal properties

In [ ]:
k  = 0.1     # Thermal conductivity
ρ  = 1270.0  # Material density
Cₚ = 600.0   # Heat capacity at constant pressure
    
l = AsteroidThermoPhysicalModels.thermal_skin_depth(P, k, ρ, Cₚ)  # Thermal skin depth
Γ = AsteroidThermoPhysicalModels.thermal_inertia(k, ρ, Cₚ)        # Thermal inertia

In [ ]:
thermo_params = AsteroidThermoPhysicalModels.thermoparams(
    P       = P,     # Rotation period
    l       = l,     # Thermal skin depth
    Γ       = Γ,     # Thermal inertia
    A_B     = 0.04,  # Bond albedo
    A_TH    = 0.0,   # Albedo in thermal infrared
    ε       = 1.0,   # Emissivity
    z_max   = 0.6,   # Maximum depth to solve the 1-D heat conduction (Lower boundary)
    Nz      = 61,    # Number of depth grid points to solve the 1-D heat conduction equation
)

Create a model for the single asteroid

In [ ]:
stpm = AsteroidThermoPhysicalModels.SingleTPM(shape, thermo_params;
    SELF_SHADOWING = true,  # Enable self-shadowing, i.e., shadowing by local topography
    SELF_HEATING   = true,  # Enable self-heating, i.e., energy exchange between interfacing facets
    SOLVER         = AsteroidThermoPhysicalModels.ForwardEulerSolver(thermo_params),  # Solver for the 1-D heat conduction equation
    BC_UPPER       = AsteroidThermoPhysicalModels.RadiationBoundaryCondition(),       # Upper boundary condition (surface layer)
    BC_LOWER       = AsteroidThermoPhysicalModels.InsulationBoundaryCondition(),      # Lower boundary condition (bottom layer)
);

In [ ]:
AsteroidThermoPhysicalModels.init_temperature!(stpm, 240);  # Intial temperature [K]

Run TPM

In [ ]:
times_to_save = ephem.time[end-nsteps_in_cycle:end]  # Save temperature during the final rotation
face_ID = [1, 2, 3, 4, 10]  # Face indices to save subsurface temperature

result = AsteroidThermoPhysicalModels.run_TPM!(stpm, ephem, times_to_save, face_ID; show_progress=false);

Save TPM result

In [ ]:
dirpath = "./output"
mkpath(dirpath)
AsteroidThermoPhysicalModels.export_TPM_results(dirpath, result)

# Data analysis and visualization

Plot the energy conservation ratio `E_cons` to check computational convergence.

`E_cons` is defined as the ratio of the output energy to the input energy from the entire surface of the asteroid over one rotation.
As the thermal calculation converges, `E_cons` approaches 1.

In [ ]:
fig = Figure()
ax = Axis(fig[1, 1],
    xlabel = "Time [h]",
    ylabel = "E_cons [-]",
)

xs = @. (ephem.time - ephem.time[begin]) / 3600  # Time since the beginning of TPM in unit of hour

hlines!(ax, [1], color=:black, linestyle=:dash)
scatterlines!(ax, xs, result.E_cons, markercolor=:blue, marker=:circle)

display(fig)

In [ ]:
plot_shape(shape; colorbar_title="Radius [m]")

In [ ]:
plot_shape(shape;
    title          = "Temperature map at the final time step.",
    colorbar_title = "Temperature [K]",
    intensity      = result.surface_temperature[:, end],
    colorscale     = :thermal,
)